<a href="https://colab.research.google.com/github/paras07/Flipkart-Reviews-Web-Scraping/blob/main/Flipkart_IPhone_14_Pro_Max_Reviews_Scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Installation of Chromium, Chromium Driver and Selenium
%%shell

cat > /etc/apt/sources.list.d/debian.list <<'EOF'
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg

cat > /etc/apt/preferences.d/chromium.pref << 'EOF'
Package: *
Pin: release a=eoan
Pin-Priority: 500


Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300


Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
EOF

# Install chromium and chromium-driver
apt-get update
apt-get install chromium chromium-driver

# Install selenium
pip install selenium

In [ ]:
from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.headless = True

In [ ]:
from bs4 import BeautifulSoup
from google.colab import  drive
drive.mount('/drive')
import requests as rq
import pandas as pd
from time import sleep
import re

In [ ]:
username=[]
rating=[]
review=[]
comment=[]
link=[]
#URL= "https://www.flipkart.com/apple-iphone-14-pro-max-space-black-128-gb/p/itm9aed88fe43457?pid=MOBGHWFHCNVGGMZF&fm=organic&ppt=dynamic&ppn=dynamic&ssid=044ccsdk1s0000001685358094251"
url=input('Enter URL: ')
driver=webdriver.Chrome('chromedriver',options=chrome_options)
driver.maximize_window()
driver.get(url)
r1=rq.get(url)
soup1=BeautifulSoup(r1.text,'html.parser')
driver.execute_script('window.scroll(0,2500)')
sleep(2)

for t in soup1.findAll('a',attrs={'href':re.compile("/product-reviews")}):
    q=t.get('href')
    link.append(q)
#print(link)
for i in link:
    if 'LSTMOBGHWFHCNVGGMZFEEIZN3' in i:
        #print(i)
        aa=i
f_url=('https://www.flipkart.com'+str(i))
i=1
#As Reviews has 11 webpages loop for 11 applied
while i<=11:
    ss=driver.get(str(f_url)+"&page="+str(i))
    qq=driver.current_url
    r2=rq.get(qq)
    soup=BeautifulSoup(r2.text,'html.parser')

#Scraping Username
    for ur in soup.find_all('p',{'class':'_2sc7ZR _2V5EHH'}):
        aa=ur.get_text()
        username.append(aa)
#Scraping Rating
    for ra in soup.find_all('div',{'class':'_3LWZlK _1BLPMq'}):
        bb=ra.get_text()
        rating.append(bb)
#Scraping Review
    for re in soup.find_all('p',{'class':'_2-N8zT'}):
        cc=re.get_text()
        review.append(cc)    
#Scraping Comment
    for co in soup.find_all('div',{'class':'t-ZTKy'}):
        dd=co.get_text()
        comment.append(dd)
 
    sleep(1)
    i+=1
    
df=pd.DataFrame([username, rating, review, comment]).transpose()
df.columns = ["USERNAME", "RATING", "REVIEW", "COMMENT"]
#File will be saved to Drive
df.to_excel("/drive/My Drive/Colab_Data/Flipkart-IPhone 14 Pro Max.xlsx") 